# Test notebook

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import folium
import streamlit as st
import os
import matplotlib
from rasterstats import zonal_stats
import rasterio
import requests

from src.utils import *
from src.utils_population import *
from src.utils_plotting import *

2023-03-20 13:46:39.651 
  command:

    streamlit run /home/daniele/miniconda3/envs/proximity/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]


## General parameters

In [3]:
verbose = True

## User input

In [4]:
# Parameters given by the user
# gdf_file = "test_data/mda_isochrones.geojson"
gdf_file = 'test_data/mda_isochrones.zip'
# gdf_file = 'test_data/shakeintensity_complex.zip'
data_type = "UNadj_constrained"
year = 2020
aggregated = False

## Process input data

In [5]:
# Create geopandas dataframes and prepare starting points
gdf, error = load_gdf(gdf_file)

if error:
    raise ValueError(error)
    
gdf = gdf.iloc[:5, ]

In [6]:
gdf

,type,value,total_pop,geometry_t,name,geometry
0,Feature,900.0,364045.0,Polygon,Juv,"POLYGON ((28.77104 47.00202, 28.77110 47.00332..."
1,Feature,1800.0,832073.0,Polygon,Juv,"POLYGON ((28.62525 47.01751, 28.62700 47.02066..."
2,Feature,2700.0,1173377.0,Polygon,Juv,"POLYGON ((28.43041 47.13080, 28.43098 47.13081..."
3,Feature,3600.0,1335679.0,Polygon,Juv,"POLYGON ((28.22984 47.11718, 28.23101 47.12058..."
4,Feature,900.0,353627.0,Polygon,Institutul Mamei ?i Copilului,"POLYGON ((28.77360 47.00231, 28.77362 47.00250..."


## Run computation

In [8]:
ee.Initialize()

In [9]:
%%time

gdf_with_pop = add_population_data(
    gdf=gdf,
    data_type=data_type,
    tif_folder='test_data/pop_data',
    data_folder='data',
    year=year,
    aggregated=aggregated,
    clobber=True,
    verbose=True,
    text_on_streamlit=False,
    progress_bar=False,
)

Retrieving data from Google Earth Engine...
Band (1/36): M_0
Band (2/36): M_1
Band (3/36): M_5
Band (4/36): M_10
Band (5/36): M_15
Band (6/36): M_20
Band (7/36): M_25
Band (8/36): M_30
Band (9/36): M_35
Band (10/36): M_40
Band (11/36): M_45
Band (12/36): M_50
Band (13/36): M_55
Band (14/36): M_60
Band (15/36): M_65
Band (16/36): M_70
Band (17/36): M_75
Band (18/36): M_80
Band (19/36): F_0
Band (20/36): F_1
Band (21/36): F_5
Band (22/36): F_10
Band (23/36): F_15
Band (24/36): F_20
Band (25/36): F_25
Band (26/36): F_30
Band (27/36): F_35
Band (28/36): F_40
Band (29/36): F_45
Band (30/36): F_50
Band (31/36): F_55
Band (32/36): F_60
Band (33/36): F_65
Band (34/36): F_70
Band (35/36): F_75
Band (36/36): F_80
Done!
CPU times: user 935 ms, sys: 31.8 ms, total: 967 ms
Wall time: 35.7 s


In [10]:
gdf_with_pop

,type,value,total_pop,geometry_t,name,geometry,pop_f_00,pop_f_01,pop_f_05,pop_f_10,...,pop_m_35,pop_m_40,pop_m_45,pop_m_50,pop_m_55,pop_m_60,pop_m_65,pop_m_70,pop_m_75,pop_m_80
0,Feature,900.0,364045.0,Polygon,Juv,"POLYGON ((28.77104 47.00202, 28.77110 47.00332...",1498,6860,10333,9556,...,15126,13048,11868,11641,11919,11547,8887,3960,2547,2503
1,Feature,1800.0,832073.0,Polygon,Juv,"POLYGON ((28.62525 47.01751, 28.62700 47.02066...",3482,15942,23846,22140,...,34984,29805,26815,25992,26976,25615,20139,9032,5855,5753
2,Feature,2700.0,1173377.0,Polygon,Juv,"POLYGON ((28.43041 47.13080, 28.43098 47.13081...",4419,20231,30112,28094,...,44409,37613,33757,32102,33838,31754,25619,11499,7562,7429
3,Feature,3600.0,1335679.0,Polygon,Juv,"POLYGON ((28.22984 47.11718, 28.23101 47.12058...",5488,25125,37207,34854,...,54889,46423,41576,38927,41867,38822,31989,14289,9574,9406
4,Feature,900.0,353627.0,Polygon,Institutul Mamei ?i Copilului,"POLYGON ((28.77360 47.00231, 28.77362 47.00250...",1454,6659,10031,9276,...,14683,12667,11522,11303,11572,11212,8628,3845,2473,2430


## Plot

In [30]:
aggregated=aggregated
joint=False
col_label = 'value'
legend_title = 'Admin level'
plot_title = 'Admin level'

fig = plot_pop_data(
    gdf=zonal_stat, 
    col_label=col_label,
    legend_title=legend_title, 
    plot_title=plot_title,
    aggregated=aggregated,
    joint=joint,
    cmap_name='viridis'
    )

fig.show()